# Using Decision Trees: detailed example

The _Play Tennis_ example data (from Mitchell (1997) _Machine Learning_) used in the notes for Week 6 is a nice, relatively small example.

The predictors are discrete-valued labels, making the decision splits easier to understand.

The first step is to read the data.

In [1]:
import math
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
df = pd.read_csv("playTennis.csv",header=0,quotechar="'")
print(df)

     outlook  temp humidity windy play
0      sunny   hot     high  fals   no
1      sunny   hot     high   tru   no
2   overcast   hot     high  fals  yes
3      rainy  mild     high  fals  yes
4      rainy  cool   normal  fals  yes
5      rainy  cool   normal   tru   no
6   overcast  cool   normal   tru  yes
7      sunny  mild     high  fals   no
8      sunny  cool   normal  fals  yes
9      rainy  mild   normal  fals  yes
10     sunny  mild   normal   tru  yes
11  overcast  mild     high   tru  yes
12  overcast   hot   normal  fals  yes
13     rainy  mild     high   tru   no


For convenience, I have created a function to evaluate the _entropy_ function `H`.

It looks more complex than it is - mainly because I assemble the calculation in a string, for teaching purposes. This string is returned to the caller and `eval`uated there by the calling program.

Note that `H` has two forms, depending on whether it is Level 1 (applies to the overall set: arguments `predNameB` and `predLabelB` are each empty strings and `pd` evaluates to the number of rows in the overall set) or Level 2 (when it is conditional on a setting of one of the predictors, specified by `predNameB` and `predLabelB` and `pd` is also more specific).

If a particular predictor setting does not arise for a given class setting, the row count `pn` is zero and the log term needs special treatment. The log of zero is minus infinity, but the log term itself is multipled by zero, so the two multiplied together is taken as zero..

In [6]:
def calcH(predNameB,predLabelB,pd):
  acc = ""
  for classLabel in labels[className]:
    pns = "rowCount[className]"+predNameB+"[classLabel]"+predLabelB
    pn = eval(pns)
    p = "("+str(pn) +"/" + str(pd)+")"
    if (pn == 0):
      acc += " -" + p + " "
    else:
      acc += " -" + p + " * math.log(" + p + ", 2) "
  return acc

## Preliminary: Deriving the row counts

We now start to count rows depending on various (combinations of) settings. We use a rowCount set to store the counts. The first dictionary element gets the count of `'ALL'` rows.

We then get a list of the column names `colNames`and the names of the class to be predicted `className`.

Looping over the column names, we ask what the unique list of labels is for each column name.

We can then loop over each of these labels and count the number of rows for which this column name takes a particular label value.

In [3]:
rowCount = {}
rowCount["ALL"] = len(df.index)
colNames = list(df.columns.values)
className = "play"
labels = {}
for colName in colNames:
  labels[colName] = df[colName].unique().tolist()
  rowCount[colName] = {}
  for label in labels[colName]:
    rowCount[colName][label] = len(df.loc[df[colName] == label].index)
pp.pprint(rowCount)

{ 'ALL': 14,
  'humidity': {'high': 7, 'normal': 7},
  'outlook': {'overcast': 4, 'rainy': 5, 'sunny': 5},
  'play': {'no': 5, 'yes': 9},
  'temp': {'cool': 4, 'hot': 4, 'mild': 6},
  'windy': {'fals': 8, 'tru': 6}}


For the Decision Tree classifier, we need to go to Level 2 also.

Therefore we also need to split the predictor counts above based on whether the decision was to play or not.

Our first step is to use a _list comprehension_ to return a list of the predictor column names only `predNames`.

We can then loop over just the predictors.

The resulting `rowCount` values depend on both the `className` = `classLabel` and the `predName` = `predLabel` conditions.

We then print the rowCount variable and see that `rowCount` has been extended with these counts.


In [4]:
predNames = [x for x in colNames if className not in x]
for predName in predNames:
  rowCount[className][predName] = {}
  for classLabel in labels[className]:
    rowCount[className][predName][classLabel] = {}
    for predLabel in labels[predName]:
      rowCount[className][predName][classLabel][predLabel] = len(df.loc[(df[className] == classLabel) & (df[predName] == predLabel)].index)
pp.pprint(rowCount)


{ 'ALL': 14,
  'humidity': {'high': 7, 'normal': 7},
  'outlook': {'overcast': 4, 'rainy': 5, 'sunny': 5},
  'play': { 'humidity': { 'no': {'high': 4, 'normal': 1},
                          'yes': {'high': 3, 'normal': 6}},
            'no': 5,
            'outlook': { 'no': {'overcast': 0, 'rainy': 2, 'sunny': 3},
                         'yes': {'overcast': 4, 'rainy': 3, 'sunny': 2}},
            'temp': { 'no': {'cool': 1, 'hot': 2, 'mild': 2},
                      'yes': {'cool': 3, 'hot': 2, 'mild': 4}},
            'windy': { 'no': {'fals': 2, 'tru': 3},
                       'yes': {'fals': 6, 'tru': 3}},
            'yes': 9},
  'temp': {'cool': 4, 'hot': 4, 'mild': 6},
  'windy': {'fals': 8, 'tru': 6}}


## Entropy calculations

The top-level entropy calculation is calculated without any splits by predictor.

It is calculated in terms of the class column ("play" in this case) which takes two values: "yes" and "no". As expected it takes a value based on how predictable this variable is.

Note that the `calcH` function is called with empty strings for `predNameB`  and `predLabelB` and the `rowCount` for ALL settings.

Also, the string returned by `calcH` needs to be evaluated to a number.

We print both the `H` expression (as a string) and its value (as a number).

In [5]:
H = {}
H[className] = {}
acc = calcH(predNameB="",predLabelB="",pd=rowCount["ALL"])
H[className][""] = eval(acc)

print(acc)
print(H[className][""])

 -(5/14) * math.log((5/14), 2)  -(9/14) * math.log((9/14), 2) 
0.9402859586706309


We now consider the effects of splitting by each of the predictors in turn.

In each case we scale by `p` which is the probability of a particular setting.

The other term is of course the `H` for that setting with the class variable. Note that here `calcH` has `predNameB` and `predLabelB` that are not just empty strings.

Using the `term` counter, we can tell whether we need to start a new accumulated string or just add to an existing accumulated string.

In [9]:
for predName in predNames:
  term = 0
  predNameB = '["'+predName+'"]'
  for predLabel in labels[predName]:
    predLabelB = '["'+predLabel+'"]'
    p = "(" + str(rowCount[predName][predLabel]) + "/" + str(rowCount["ALL"]) + ") * "
    a = p + "("+calcH(predNameB,predLabelB,pd=rowCount[predName][predLabel])+")"
    if (term == 0):
      acc = a
    else:
      acc = acc + " + " + a
    term += 1
  H[className][predName] = eval(acc)

  print(acc)
  print("Splitting on {} gives entropy {}".format(predName,H[className][predName]))


(5/14) * ( -(3/5) * math.log((3/5), 2)  -(2/5) * math.log((2/5), 2) ) + (4/14) * ( -(0/4)  -(4/4) * math.log((4/4), 2) ) + (5/14) * ( -(2/5) * math.log((2/5), 2)  -(3/5) * math.log((3/5), 2) )
Splitting on outlook gives entropy 0.6935361388961918
(4/14) * ( -(2/4) * math.log((2/4), 2)  -(2/4) * math.log((2/4), 2) ) + (6/14) * ( -(2/6) * math.log((2/6), 2)  -(4/6) * math.log((4/6), 2) ) + (4/14) * ( -(1/4) * math.log((1/4), 2)  -(3/4) * math.log((3/4), 2) )
Splitting on temp gives entropy 0.9110633930116763
(7/14) * ( -(4/7) * math.log((4/7), 2)  -(3/7) * math.log((3/7), 2) ) + (7/14) * ( -(1/7) * math.log((1/7), 2)  -(6/7) * math.log((6/7), 2) )
Splitting on humidity gives entropy 0.7884504573082896
(8/14) * ( -(2/8) * math.log((2/8), 2)  -(6/8) * math.log((6/8), 2) ) + (6/14) * ( -(3/6) * math.log((3/6), 2)  -(3/6) * math.log((3/6), 2) )
Splitting on windy gives entropy 0.8921589282623617


So which predictor should we use in our node, to split the data to get the smallest entropy over the available predictors?

It seems that the `outlook` variable is the one to choose!

In [14]:
splitVariable = min(H[className], key=H[className].get)
print("The first split should be on the _{}_ variable, reducing the entropy from {} to {}".format(splitVariable,H[className][""],H[className][splitVariable]))


The first split should be on the _outlook_ variable, reducing the entropy from 0.9402859586706309 to 0.6935361388961918


Note that this process can be continued as necessary, with it stopping when all leafs are _pure_ (have entropy = 0). According to https://codefying.com/2015/03/09/decision-tree-classifier-part-1/, the resulting decision tree is shown below:

![View of Play Tennis decision tree](https://codefying.files.wordpress.com/2015/03/mltree.jpg)

Note that the `temp` predictor is not used in the decision tree - it is not needed.

## Using sklearn to derive the decision tree

Note that the treatment above is intended to help understanding. There is no need to program Decision Tree classifiers yourself!

`scikit-learn` offers a `DecisionTreeClassifier` with the same sort of API as other supervised learning algorithms, alongside settings that are specific to Decision Trees.

One awkward feature of the sklearn implementation is that it seems to be necessary to code the labels as integers. While there are tools to do this, it does mean that the resulting decision tree is not as easy to "read"